### Imports

In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

### Recolha de dados

In [13]:
# 64 patients with breast cancer and 52 healthy controls, 116 in total
# 1 - paciente sem cancro, 2 - paciente com cancro

file = pd.read_excel("../Data/dataR2.XLSX")
DataSet = pd.DataFrame(data=file.dropna()) #Remove NaN because of initial empty lines

# Find the indexes where the differente class samples are located
ixHealthy = np.where(DataSet['Classification'] == 1)
ixWithCancer = np.where(DataSet['Classification'] == 2)

# Definir classes
Classes = ["healthy","withCancer"]


DataSet['Classification'].iloc[ixHealthy] = Classes[0] 
DataSet['Classification'].iloc[ixWithCancer] = Classes[1] 

#for w in DataSet.columns:
#    for i in DataSet.columns: 
#        if w == 'Classification' or i == 'Classification' or w == i:
#            continue
#        fig = px.scatter_3d(x = DataSet.index, y = DataSet[w], z = DataSet[i], color = DataSet['Classification'], labels=dict(x = "N", y = w, z = i , color = "Classification"))
#        #fig = px.scatter_3d(x = CorkStopp["N.1"], y = CorkStopp["ARM"],z = CorkStopp["PRM"],color = CorkStopp['C'], labels=dict(x = "N", y = "ARM", color = "Cork Stoppers"))
#
#        fig.update_traces(marker_size=10)
#        fig.show()

# Partição
x = DataSet.iloc[:, :-1] # todas menos a última coluna

y = DataSet.iloc[:, -1] # apenas a última coluna


x_train, x_testInicial, y_train, y_testInicial = train_test_split( # Partições entre train e test
    DataSet, y,
    test_size=0.2,       # 20% dos dados para teste
    random_state=67,     # garante reprodutibilidade
    shuffle=True,
    stratify = y           # mantém a proporção das classes TODO: se calhar tirar depois
)


x_val, x_test, y_val, y_test = train_test_split( # Partições entre test e validation
    x_testInicial, y_testInicial,
    test_size=0.5,
    random_state=67,
    stratify=y_testInicial  # mantém a proporção das classes TODO: se calhar tirar depois
)



C:\Users\pedro\AppData\Local\Temp\ipykernel_3084\1934334510.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Relevancy test

In [14]:
# Kruskall

X= x.to_numpy()
X=X.astype(float)

fnames=x.columns

Hs={}
for i in range(np.shape(X)[1]):
    st=stats.kruskal(X[ixHealthy,i].flatten(),X[ixWithCancer,i].flatten())
    Hs[fnames[i]]=st.statistic


Hs = sorted(Hs.items(), key=lambda x: x[1],reverse=True)  

print("Ranked features")

for f in Hs:
    print(f[0]+"-->"+str(f[1]))

#print(X)


Ranked features
Glucose-->23.91957997825514
Resistin-->9.699241863905343
HOMA-->8.887111686390483
Insulin-->4.931023337417101
BMI-->1.6374345259560847
Age-->0.5053505863773902
MCP.1-->0.4512315968813051
Adiponectin-->0.08986720937572039
Leptin-->0.004437869822481844


### Redundancy test

In [ ]:

# X = np.array([x[h[0]] for h in Hs])

# # Nomes das features
# labels = [h[0] for h in Hs]

# corrMat=np.corrcoef(X)
# fig = px.imshow(corrMat, text_auto=True,labels=dict(x="Features", y="Features", color="Correlation"),
#                 x=labels,
#                 y=labels,
#                 width=800, height=800,color_continuous_scale=px.colors.sequential.gray)
# fig.show()


firstFive = Hs[:]
corr = x[[f[0] for f in firstFive]].corr()
print(corr)


# HOMA - Insulin 0.932 Muito alta -> redundância clara


              Glucose  Resistin      HOMA   Insulin       BMI       Age  \
Glucose      1.000000  0.291327  0.696212  0.504653  0.138845  0.230106   
Resistin     0.291327  1.000000  0.231101  0.146731  0.195350  0.002742   
HOMA         0.696212  0.231101  1.000000  0.932198  0.114480  0.127033   
Insulin      0.504653  0.146731  0.932198  1.000000  0.145295  0.032495   
BMI          0.138845  0.195350  0.114480  0.145295  1.000000  0.008530   
Age          0.230106  0.002742  0.127033  0.032495  0.008530  1.000000   
MCP.1        0.264879  0.366474  0.259529  0.174356  0.224038  0.013462   
Adiponectin -0.122121 -0.252363 -0.056337 -0.031296 -0.302735 -0.219813   
Leptin       0.305080  0.256234  0.327210  0.301462  0.569593  0.102626   

                MCP.1  Adiponectin    Leptin  
Glucose      0.264879    -0.122121  0.305080  
Resistin     0.366474    -0.252363  0.256234  
HOMA         0.259529    -0.056337  0.327210  
Insulin      0.174356    -0.031296  0.301462  
BMI          0